In [4]:
import json, sys, uuid, hashlib, ast, requests, os
from PIL import Image
from sqlalchemy import create_engine, MetaData
from sqlalchemy.sql import text
from llama_index.core import SQLDatabase
from llama_index.core.schema import TextNode, ImageNode
import urllib.parse
from tqdm import tqdm
# from Retrieval_Server.modules.load_images import load_svsortiff_image, load_pngorjpg_image
from sqlalchemy.sql import text
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
def create_text_node_from_sql(sql_database, uuid_namespace):
    nodes = []
    with sql_database.engine.connect() as connection:
        
        tables = connection.execute(text("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")).fetchall()
        
        for table in tables:
            print("table", table)
            table_name = table[0]
            
            contents = connection.execute(text(f"SELECT * FROM {table_name}")).fetchall()
            print("contents", contents)
            if len(contents) == 0:
                continue
            

            for row in contents:
                print("row", row)
                break
                row_dict = {}
                for i in range(len(row)):
                    row_dict[str(i)] = str(row[i])
                nodes.append(TextNode(id=uuid.uuid5(uuid_namespace, str(row_dict)),
                                    text=str(row_dict),
                                    metadata=row_dict))
    return nodes

In [ ]:
user_name, password, ip, port, db_name = r'aladdin_remap_ro', r'[rFPe!8|b(~lrnADYI91r>H:@S~z3x', "localhost", "5432", "aladdin_remap"
uuid_namespace = uuid.NAMESPACE_URL
encoded_password = urllib.parse.quote_plus(password)
print(f'postgresql://{user_name}:{encoded_password}@{ip}:{port}/{db_name}')

engine = create_engine(f'postgresql://{user_name}:{encoded_password}@{ip}:{port}/{db_name}')
metadata = MetaData()

metadata.create_all(engine)

sql_database = SQLDatabase(engine)
text_nodes = create_text_node_from_sql(sql_database, uuid_namespace)